In [1]:
import jsonpickle

import json
import pprint

def json_pretty_print(parsed):
    if isinstance(parsed, str):
        parsed = json.loads(parsed)
    #ret = pprint.pformat(parsed)
    ret = json.dumps(parsed, indent=4, sort_keys=True)
    return ret

import jsonpickle.ext.pandas as jsonpickle_pandas
jsonpickle_pandas.register_handlers()

In [46]:
import inspect

def round_trip(obj):
    print("obj=\n", obj)
    frozen = jsonpickle.encode(obj)
    #
    print("frozen=")
    print(json_pretty_print(frozen))
    #
    obj2 = jsonpickle.decode(frozen)
    print("obj2=\n", obj2)
    print("class=%s" % type(obj))
    if str(type(obj)).startswith("<class '"):
        pass
    else:
        if isinstance(obj, pd.DataFrame):
            assert obj.equals(obj2)
        else:
            assert obj == obj2
    return obj2

In [22]:
import pandas as pd

data = {
    'Product': ['Desktop Computer', 'Tablet', 'iPhone', 'Laptop'],
    'Price': [700, 250, 800, 1200]
}

df = pd.DataFrame(data, columns=['Product', 'Price'])
df.index.name = "hello"
print(df)

                Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200


In [23]:
df2 = round_trip(df)

obj=
                 Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200
frozen=
{
    "meta": {
        "dtypes": {
            "Price": "int64",
            "Product": "object"
        },
        "index": "{\"py/object\": \"pandas.core.indexes.range.RangeIndex\", \"values\": \"[0, 1, 2, 3]\", \"txt\": true, \"meta\": {\"dtype\": \"int64\", \"name\": \"hello\"}}"
    },
    "py/object": "pandas.core.frame.DataFrame",
    "txt": true,
    "values": "Product,Price\nDesktop Computer,700\nTablet,250\niPhone,800\nLaptop,1200\n"
}
obj2=
                 Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200
<class 'pandas.core.frame.DataFrame'>


In [47]:
class Thing:
    def __init__(self, name):
        self.name = name
        
obj = Thing('Awesome')

round_trip(obj)

obj=
frozen=
{
    "name": "Awesome",
    "py/object": "__main__.Thing"
}
obj2=
class=<class '__main__.Thing'>


In [ ]:
a = 

In [123]:
def test(a: int, b: int):
    print(round_trip(a))

    
test("strunz", 6)
test(4, 6)
test(["hello"], 6)

obj=
 strunz
frozen=
"strunz"
obj2=
 strunz
strunz
obj=
 4
frozen=
4
obj2=
 4
4
obj=
 ['hello']
frozen=
[
    "hello"
]
obj2=
 ['hello']
['hello']


In [55]:
df.index.dtype#

dtype('int64')

In [52]:
df.dtypes

Product    object
Price       int64
dtype: object

In [58]:
#import io
#import io.StringIO
#from io import StringIO

#output = StringIO.StringIO()

orient = "columns"
#orient = "split"
#orient = "records"
#orient = "table"
df_as_str = df.to_json(orient=orient)

# split
# records
# index
# values
# table
# columns (the default format)

python_code = []
target_var = "df_as_str"
python_code.append("%s = %s" % (target_var, df_as_str))
python_code.append("%s.index.name = %s" % (target_var, df.index.name))
python_code = "\n".join(python_code)
print(python_code)

exec(python_code)

df_as_str = {"Product":{"0":"Desktop Computer","1":"Tablet","2":"iPhone","3":"Laptop"},"Price":{"0":700,"1":250,"2":800,"3":1200}}
df_as_str.index.name = hello


NameError: name 'hello' is not defined

In [33]:
arr = eval(df_as_str)
df2 = pd.DataFrame.from_dict(arr, orient="columns")
df2.index.name

,Product,Price
0,Desktop Computer,700
1,Tablet,250
2,iPhone,800
3,Laptop,1200
